In [2]:
import cv2
import face_recognition
import os
import numpy as np
import warnings
import pickle 
import datetime
import time 
from collections import deque 

# IMPORTANT: Ensure 'utils.py' is in your directory and contains these functions.
from utils import (
    load_encodings, load_mask_detector, load_liveness_detector, 
    detect_mask, detect_liveness_ear, log_attendance, draw_info_on_frame, 
    save_encodings, load_encryption_key,
    landmark_history
)

warnings.filterwarnings("ignore", category=UserWarning)

# --- Configuration Paths ---
DATASET_PATH = "dataset"
ENCODINGS_FILE = 'encrypted_data/face_encodings.pkl' # Path to save the encrypted encodings

# --- NEW: List of folder names to exclude from face encoding (e.g., large datasets) ---
# FIX APPLIED: 'fmd_training_data' is now explicitly excluded.
EXCLUDED_FOLDERS = ['archive', 'train', 'test', 'validation', 'raw_images', 'mask_data', 'fmd_training_data']

known_encodings = []
known_names = []
perform_encoding = True

# Ensure the encryption key is loaded
load_encryption_key()

# --- Check if encodings already exist and load them ---
if os.path.exists(ENCODINGS_FILE):
    print("Found existing encrypted encodings file. Attempting to load...")
    known_encodings, known_names = load_encodings(ENCODINGS_FILE) 
    
    if len(known_encodings) > 0:
        print(f"Successfully loaded {len(known_encodings)} known faces.")
        perform_encoding = False # This line prevents re-encoding every time
        
        # --- INSTRUCTIONAL MESSAGE ---
        print("\nNOTE: Encoding skipped to save time.")
        print("To add new faces, you MUST delete the file:")
        print(f"      {ENCODINGS_FILE}")
        print("      ... and then rerun this cell.")
        # ---------------------------
        
    else:
        print("Loaded encodings file was empty or corrupted. Starting fresh encoding.")


# --- Encoding Script (Only runs if file doesn't exist or loading failed) ---
if perform_encoding:
    print("Starting data encoding from dataset...")
    
    if not face_recognition:
        print("ERROR: face_recognition library is required for encoding.")
        
    for person_name in os.listdir(DATASET_PATH):
        person_folder = os.path.join(DATASET_PATH, person_name)
        
        # --- NEW EXCLUSION CHECK: This skips the bulk folders ---
        if person_name.lower() in EXCLUDED_FOLDERS:
             print(f"Skipping excluded bulk dataset folder: {person_name}")
             continue
        # ---------------------------
        
        if not os.path.isdir(person_folder) or person_name.startswith('.'):
            continue

        for filename in os.listdir(person_folder):
            img_path = os.path.join(person_folder, filename)
            
            if os.path.isdir(img_path) or not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue
            
            try:
                image = face_recognition.load_image_file(img_path)
                encodings = face_recognition.face_encodings(image, num_jitters=1) 
                
                if len(encodings) > 0:
                    known_encodings.append(encodings[0])
                    known_names.append(person_name)
                else:
                    print(f"Warning: No face found in {img_path}")
            except Exception as e:
                print(f"Skipping {img_path}: {e}")
    
    if len(known_encodings) > 0:
        save_encodings(known_encodings, known_names, ENCODINGS_FILE) 
        print(f"\nSUCCESS: Saved {len(known_encodings)} new encodings to the encrypted file.")
    else:
        print("❌ No faces found in the dataset. Please check your 'dataset' folder.")

Found existing encrypted encodings file. Attempting to load...
Loaded 9 known faces from encodings file.
Successfully loaded 9 known faces.

NOTE: Encoding skipped to save time.
To add new faces, you MUST delete the file:
      encrypted_data/face_encodings.pkl
      ... and then rerun this cell.


In [3]:
# Cell 2: Model Loading and Global Tracking Setup

# --- 1. Load the Detection Models ---
print("Loading detection models...")

# *** FIX: We are skipping the mask model load because of persistent file corruption/incompatibility. ***
# mask_loaded = load_mask_detector() 
mask_loaded = False 
print("Mask detection skipped to bypass model loading error.")

liveness_loaded = load_liveness_detector() 

if not mask_loaded:
    print("WARNING: Mask detection model failed to load. Mask checking will be skipped or unreliable.")
    # --- DEBUG INSTRUCTIONS FOR MASK ERROR ---
    print("DEBUG: Mask model check is now gracefully disabled.")
    print("ACTION REQUIRED: Proceed to the next cell. The attendance system will run without mask detection.")
    # ----------------------------------------
    
# --- 2. Global Tracking Variables ---
# Global variables to track who has already checked in during this session
checked_in_today = {}

# 1. Name Stabilization
name_history = []
MAX_HISTORY = 10  # Number of frames to check for majority vote

# 2. Unknown Person Capture
UNKNOWN_CAPTURE_COUNT = 0 # Tracks frames the unknown face is visible
UNKNOWN_CAPTURE_THRESHOLD = 30 # Number of frames before capturing (e.g., 1 second at 30 FPS)
CAPTURE_IN_PROGRESS = False # Flag to prevent repeated capture attempts
NEW_PERSON_NAME = "new_person_" + datetime.datetime.now().strftime("%Y%m%d%H%M%S") # Unique ID for the new person

# ----------------- VERIFICATION STEP -----------------
# known_encodings and known_names are defined in Cell 1 and accessible here.
print(f"Total known faces loaded: {len(known_encodings)}")
if len(known_names) > 0:
    print(f"Last loaded name: {known_names[-1]}")
    # IMPORTANT: Replace '[New Name]' with the name of the folder you added (e.g., 'david')
    print(f"Is your new person's name in the list? {'[New Name]' in known_names}")


Loading detection models...
Mask detection skipped to bypass model loading error.
Using EAR-based Liveness detection; skipping Liveness Deep Model loading.
DEBUG: Mask model check is now gracefully disabled.
ACTION REQUIRED: Proceed to the next cell. The attendance system will run without mask detection.
Total known faces loaded: 9
Last loaded name: Jatin Mehra
Is your new person's name in the list? False


In [4]:
# Cell 3: Main Video Capture and Recognition Loop

# --- Video Capture Setup ---
video_capture = cv2.VideoCapture(0)

# --- Loop Control (Ensure these variables are initialized in Cell 2 or before the loop) ---
RECOGNITION_TOLERANCE = 0.45 
ATTENDANCE_COOLDOWN = 600 # 10 minutes (600 seconds)
FRAME_SKIP = 2 # Process every 2nd frame for speed
frame_count = 0 

# History variables for name stabilization (Ensure name_history is an empty list in Cell 2)
# name_history = [] 
MAX_HISTORY = 10  # Number of frames to check

# Unknown Capture variables (Ensure these are initialized in Cell 2)
# UNKNOWN_CAPTURE_COUNT = 0 
# UNKNOWN_CAPTURE_THRESHOLD = 30
# CAPTURE_IN_PROGRESS = False
# NEW_PERSON_NAME = "new_person_" + datetime.datetime.now().strftime("%Y%m%d%H%M%S") 

# --- Main Processing Loop ---
while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    frame_count += 1
    
    # 1. Image Preprocessing (downscaling for faster processing)
    if frame_count % FRAME_SKIP == 0:
        
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

        # 2. Face Detection, Encoding, AND LANDMARKS
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
        face_landmarks_list = face_recognition.face_landmarks(rgb_small_frame, face_locations)
        
        # Add 'None' to history if no face is detected, maintaining history consistency
        if len(face_landmarks_list) == 0:
            landmark_history.append(None)


        # 3. Process Each Detected Face
        for face_location, face_encoding, face_landmarks in zip(face_locations, face_encodings, face_landmarks_list):
            
            # Scale back up face location
            top, right, bottom, left = [coord * 4 for coord in face_location]
            face_roi = frame[top:bottom, left:right] 
            
            # --- Essential ROI Preprocessing for Detection Models ---
            processed_roi = None
            if face_roi.size > 0 and face_roi.shape[0] > 0 and face_roi.shape[1] > 0:
                processed_roi = cv2.resize(face_roi, (224, 224))
                processed_roi = cv2.cvtColor(processed_roi, cv2.COLOR_BGR2RGB)
                processed_roi = processed_roi.astype("float32") / 255.0
            else:
                continue 
            # ----------------------------------------------------

            current_frame_name = "Unknown" # Temporary name for this frame
            is_live = False
            is_masked = False
            
            # --- A. Face Recognition ---
            if len(known_encodings) > 0:
                face_distances = face_recognition.face_distance(known_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)
                
                if face_distances[best_match_index] <= RECOGNITION_TOLERANCE:
                    current_frame_name = known_names[best_match_index]
            
            # --- NAME STABILIZATION (Majority Vote) ---
            
            # 1. Add current name to history
            name_history.append(current_frame_name)
            
            # 2. Trim history
            if len(name_history) > MAX_HISTORY:
                name_history.pop(0) 
            
            # 3. Determine STABLE NAME via majority vote
            from collections import Counter
            stable_name = Counter(name_history).most_common(1)[0][0]
            
            # --- B. Liveliness and Masking Detection (Uses stable_name) ---
            if stable_name != "Unknown":
                # Check Liveliness (Uses landmarks)
                is_live, _ = detect_liveness_ear(face_landmarks) 
                
                if is_live:
                    # Check Masking (Uses processed_roi)
                    is_masked, _ = detect_mask(processed_roi) 
            
            # --- C. Attendance Logging (Uses stable_name) ---
            if stable_name != "Unknown" and is_live and not is_masked:
                current_time = time.time() 
                
                if (stable_name not in checked_in_today or 
                    (current_time - checked_in_today.get(stable_name, 0)) > ATTENDANCE_COOLDOWN):
                    
                    log_attendance(stable_name, "Present") 
                    checked_in_today[stable_name] = current_time
                    print(f"ATTENDANCE LOGGED for: {stable_name}")

            # --- E. NEW: Unknown Person Capture Logic ---
            if stable_name == "Unknown":
                if not CAPTURE_IN_PROGRESS:
                    UNKNOWN_CAPTURE_COUNT += 1
                    
                    # Draw countdown feedback
                    countdown_text = f"Hold still! Capturing in {UNKNOWN_CAPTURE_THRESHOLD - UNKNOWN_CAPTURE_COUNT}"
                    cv2.putText(frame, countdown_text, (left, top - 10), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 165, 255), 2)
                    
                    if UNKNOWN_CAPTURE_COUNT >= UNKNOWN_CAPTURE_THRESHOLD:
                        CAPTURE_IN_PROGRESS = True
                        
                        # --- Save the image ---
                        new_person_folder = os.path.join(DATASET_PATH, NEW_PERSON_NAME)
                        os.makedirs(new_person_folder, exist_ok=True)
                        
                        # Capture and save the face ROI for better quality training data
                        img_name = os.path.join(new_person_folder, f"{NEW_PERSON_NAME}_face_1.jpg")
                        cv2.imwrite(img_name, face_roi) 
                        
                        print(f"Captured new unknown face: {img_name}")
                        
                        # Reset for next unknown user
                        UNKNOWN_CAPTURE_COUNT = 0
                        
            else:
                # Reset capture logic if a known person is consistently detected
                UNKNOWN_CAPTURE_COUNT = 0
                if CAPTURE_IN_PROGRESS:
                    # Generate a new unique ID for the next unknown person
                    NEW_PERSON_NAME = "new_person_" + datetime.datetime.now().strftime("%Y%m%d%H%M%S")
                    CAPTURE_IN_PROGRESS = False

            # --- D. Drawing Information (Uses stable_name) ---
            frame = draw_info_on_frame(frame, (top, right, bottom, left), stable_name, is_live, is_masked)

    # Display video
    cv2.imshow("Face Attendance System - Press 'q' to Quit", frame)

    # Quit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# --- Cleanup ---
video_capture.release()
cv2.destroyAllWindows()

ATTENDANCE LOGGED for: abhinav


In [5]:
# Cell 4: Verify Attendance Log

import pandas as pd
import os

LOG_FILE = 'attendance/attendance_log.csv'

print("--- Attendance Log Summary ---")

if os.path.exists(LOG_FILE):
    try:
        # Read the CSV file into a Pandas DataFrame
        df = pd.read_csv(LOG_FILE)
        
        # Display the last few entries
        print("\nLast 10 Attendance Entries:")
        print(df.tail(10))
        
        # Display a simple summary
        print("\n--- Summary Statistics ---")
        
        # Count unique people present
        unique_attendees = df['Name'].nunique()
        print(f"Total Unique Attendees Logged: {unique_attendees}")
        
        # Show the count of records per person
        print("\nAttendance Counts per Person:")
        print(df['Name'].value_counts())
        
    except pd.errors.EmptyDataError:
        print(f"The attendance log file ({LOG_FILE}) is empty.")
    except Exception as e:
        print(f"An error occurred while reading the log file: {e}")
else:
    print(f"Attendance log file not found at: {LOG_FILE}. Ensure Cell 3 was run and recognized a person.")

--- Attendance Log Summary ---

Last 10 Attendance Entries:
             Timestamp         Name   Status
0  2025-11-15 00:54:42      abhinav  Present
1  2025-11-15 02:58:44      abhinav  Present
2  2025-11-15 03:05:27      abhinav  Present
3  2025-11-15 03:15:16      abhinav  Present
4  2025-11-15 09:52:08      abhinav  Present
5  2025-11-15 09:55:23      abhinav  Present
6  2025-11-15 10:40:41      abhinav  Present
7  2025-11-15 10:41:04  Jatin Mehra  Present
8  2025-11-15 11:31:08      abhinav  Present
9  2025-11-18 10:10:23      abhinav  Present

--- Summary Statistics ---
Total Unique Attendees Logged: 2

Attendance Counts per Person:
Name
abhinav        9
Jatin Mehra    1
Name: count, dtype: int64
